<a href="https://colab.research.google.com/github/userdarius/ml-experiments/blob/main/AlexNetMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install torchinfo
# 3rd party
import numpy as np

# We import PyTorch and some of its internal modules
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor, Lambda, Compose
from torchinfo import summary
import matplotlib.pyplot as plt

# Project files.
from helpers import accuracy, DrawingPad

In [14]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
batch_size = 128

# Dataset and DataLoader for MLP.
dataset_train = MNIST('data', train=True, download=True, transform=ToTensor())
dataset_test = MNIST('data', train=False, download=True, transform=ToTensor())
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

print('Loaded {} train and {} valid samples.'.format(len(dataset_train), len(dataset_test)))

Loaded 60000 train and 10000 valid samples.


In [67]:
# Define the loss fuction.
criterion = nn.CrossEntropyLoss()

In [68]:
def train_model(model, criterion, optimizer, dataloader_train, dataloader_test, epochs):
    """ Trains the model for the specified number of epochs on the dataset.

    Args:
        model: The model to train.
        criterion: The loss function.
        optimizer: The optimizer to use.
        dataloader_train: The DataLoader for the training set.
        dataloader_test: The DataLoader for the test set.
        epochs: The number of epochs to train for.
    """
    for ep in range(epochs):
        # Training.
        model.train()
        for it, batch in enumerate(dataloader_train):

            # 5.1 Load a batch, break it down in images and targets.
            x, y = batch

            # load to cuda
            if torch.cuda.is_available():
              x = x.cuda()  # Move inputs to GPU
              y = y.cuda()  # Move labels to GPU

            # 5.2 Run forward pass.
            logits = model(x)

            # 5.3 Compute loss (using 'criterion').
            loss = criterion(logits, y)

            # 5.4 Run backward pass.
            loss.backward()

            # 5.5 Update the weights using 'optimizer'.
            optimizer.step()

            # 5.6 Zero-out the accumulated gradients.
            optimizer.zero_grad()

            print('\rEp {}/{}, it {}/{}: loss train: {:.2f}, accuracy train: {:.2f}'.
                  format(ep + 1, epochs, it + 1, len(dataloader_train), loss,
                         accuracy(logits, y)), end='')

        # Validation.
        model.eval()
        with torch.no_grad():
            acc_run = 0
            for it, batch in enumerate(dataloader_test):
                # Get batch of data.
                x, y = batch
                if torch.cuda.is_available():
                  x = x.cuda()
                  y = y.cuda()
                curr_bs = x.shape[0]
                acc_run += accuracy(model(x), y) * curr_bs
            acc = acc_run / len(dataloader_test.dataset)

            print(', accuracy test: {:.2f}'.format(acc))

In [69]:
# Simplified AlexNet implementation for MNIST
class alexNet(nn.Module):
    """ Build your own model.
    It should take as input images of shape (1, 28, 28).
    """
    def __init__(self):
        """
        Initialize your model.

        Feel free to add argument if you want to.
        """
        super(alexNet, self).__init__()

        self.feature = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d( kernel_size=2, stride=2),
            nn.Conv2d(64, 96, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(96, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d( kernel_size=2, stride=1),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(32*12*12,2048),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(2048,1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024,10),

        )


    def forward(self, x):
        """
        Define the forward pass of your model.

        Args:
        x: torch.Tensor of shape (batch_size, 1, 28, 28)

        Returns:
        torch.Tensor of shape (batch_size, 10)
        """
        x = self.feature(x)
        x = x.view(-1,32*12*12)
        x = self.classifier(x)
        return x



In [70]:
# Instantiate the model and print its architecture.
model = alexNet()
summary(model, input_size=(1, 1, 28, 28))

# Train your model.
epochs = 5
learning_rate = 1e-1

if torch.cuda.is_available():
    model.cuda()

optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

train_model(model, criterion, optimizer, dataloader_train, dataloader_test, epochs)

train_losses = []
test_losses =[]
test_accuracy = []

# Add graphs

Ep 1/5, it 469/469: loss train: 2.29, accuracy train: 0.18, accuracy test: 0.22
Ep 2/5, it 469/469: loss train: 0.13, accuracy train: 0.95, accuracy test: 0.97
Ep 3/5, it 469/469: loss train: 0.12, accuracy train: 0.96, accuracy test: 0.98
Ep 4/5, it 469/469: loss train: 0.07, accuracy train: 0.98, accuracy test: 0.99
Ep 5/5, it 469/469: loss train: 0.02, accuracy train: 0.99, accuracy test: 0.99
